<a href="https://colab.research.google.com/github/vishwajeet-hogale/IPO-rep/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

In [ ]:
!pip install ./transformers
!pip install tensorboardX


In [ ]:
!mkdir dataset \
&& cd dataset \
&&   \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
# !export SQUAD_DIR=/content/dataset \
# && python transformers/examples/run_squad.py \
#   --model_type albert \
#   --model_name_or_path albert-base-v2 \
#   --do_train \
#   --do_eval \
#   --do_lower_case \
#   --train_file $SQUAD_DIR/train-v2.0.json \
#   --predict_file $SQUAD_DIR/dev-v2.0.json \
#   --per_gpu_train_batch_size 12 \
#   --learning_rate 3e-5 \
#   --num_train_epochs 1.0 \
#   --max_seq_length 384 \
#   --doc_stride 128 \
#   --output_dir /content/model_output \
#   --save_steps 1000 \
#   --threads 4 \
#   --version_2_with_negative 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
print(torch.__version__)


In [ ]:

from transformers import (
    

    BertTokenizer,
    BertForQuestionAnswering,
    squad_convert_examples_to_features
)


from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model



tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [ ]:
#we have to read csv here!
context = "Incorporated in 1999, MTAR Technologies is a leading national player in the precision engineering industry. The company is primarily engaged in the manufacturing of mission-critical precision components with close tolerance and in critical assemblies through its precision machining, assembly, specialized fabrication, testing, and quality control processes. Since its inception, MTAR Technologies has significantly expanded its product portfolio including critical assemblies i.e. Liquid propulsion engines to GSLV Mark III, Base Shroud Assembly & Airframes for Agni Programs, Actuators for LCA, power units for fuel cells, Fuel machining head, Bridge & Column, Drive Mechanisms, Thimble Package, etc. A wide range of complex product portfolios meets the varied requirements of the Indian nuclear, Defence, and Space sector. ISRO, NPCIL, DRDO, Bloom Energy, Rafael, Elbit, etc. are some of the esteem clients. Currently, the firm has 7 state-of-the-art manufacturing facilities in Hyderabad, Telangana that undertake precision machining, assembly, specialized fabrication, brazing and heat treatment, testing and quality control, and other specialized processes."
questions = ["which company is going to be listed?","which company is going to be IPO?","which company is going to be public?","which company is going to be listed on stock exchange?","which company is about to launch initial public offering?","which company is about to launch IPO?","which company is it talking about?","where is the company located?","in which country is it?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
import statistics as st
prelist = [i for i in predictions.values() if i != '']
try:
  print(st.mode(prelist))
except:
  print(prelist)